In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Installing required libraries
! pip install torch torchvision tensorboardx

In [0]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [0]:
def mnist_data():
  
  compose = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  out_dir = './dataset'
  
  return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True) 

In [0]:
data = mnist_data()

# Creating data loader
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)

num_batches = len(data_loader)

In [0]:
# Discriminator Class

class DiscriminatorNet(torch.nn.Module):
  
  def __init__(self):
    
    super(DiscriminatorNet, self).__init__()
    n_features = 784
    n_out = 1
    
    self.hidden0 = nn.Sequential(nn.Linear(n_features, 1024), nn.LeakyReLU(0.2), nn.Dropout(0.3))
    
    self.hidden1 = nn.Sequential(nn.Linear(1024, 512), nn.LeakyReLU(0.2), nn.Dropout(0.3))
    
    self.hidden2 = nn.Sequential(nn.Linear(512, 256), nn.LeakyReLU(0.2), nn.Dropout(0.3))
    
    self.out = nn.Sequential(nn.Linear(256, n_out), nn.LeakyReLU(0.2), nn.Dropout(0.3))
    
    
  def forward(self, x):
    
    x = self.hidden0(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.out(x)
    return x
    

In [0]:
# Object of Discriminator class initialised

discriminator = DiscriminatorNet()

In [0]:
# Functions to convert images to vectors and vice-versa

def images_to_vectors(images):
  return images.view(images.size(0), 784)

def vectors_to_images(vectors):
  return vectors.view(vectors.size(0), 1, 28, 28)

In [0]:
# Generator Class

class GeneratorNet(torch.nn.Module):
  
  def __init__(self):
    
    super(GeneratorNet, self).__init__()
    n_features = 100
    n_out = 784
    
    self.hidden0 = nn.Sequential(nn.Linear(n_features, 256), nn.LeakyReLU(0.2))
    
    self.hidden1 = nn.Sequential(nn.Linear(256, 512), nn.LeakyReLU(0.2))
    
    self.hidden2 = nn.Sequential(nn.Linear(512, 1024), nn.LeakyReLU(0.2))
    
    self.out = nn.Sequential(nn.Linear(1024, n_out), nn.Tanh())
    
  
  def forward(self, x):
    
    x = self.hidden0(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.out(x)
    
    return x


In [0]:
# Generator Object Initialised
generator = GeneratorNet()

In [0]:
# Introducing some random noise sampled from a normal distribution 
# with mean 0 and variance 1

def noise(size):
  
  n = Variable(torch.randn(size, 100))
  
  return n

In [0]:
# Optimizers for both generator and discriminator

d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [0]:
# Binary Cross Entropy Loss Function

loss = nn.BCELoss()

In [0]:
def ones_target(size):
  
  data = Variable(torch.ones(size, 1))
  
  return data

def zeroes_target(size):
  
  data = Variable(torch.zeros(size, 1))
  
  return data

In [0]:
# Training Discriminator

def  train_discriminator(optimizer, real_data, fake_data):
  
  N = real_data.size(0)
  optimizer.zero_grad()  # Reset Gradients
  
  # Training on Real Data
  
  prediction_real = discriminator(real_data)
  error_real = loss(prediction_real, ones_target(N)) # Calculating Error
  error_real.backward()   # Backpropagation
  
  
  # Training on Fake Data
  
  prediction_fake = discriminator(fake_data)
  error_fake = loss(prediction_fake, zeroes_target(N))  # Calculating Error
  error_fake.backward()    # Backpropagation
  
  
  # Updating Weights
  
  optimizer.step()
  
  return error_real + error_fake, prediction_real, prediction_fake
  